In [1]:
import os
import glob
import sqlite3
import pandas as pd
import json
import numpy as np

In [2]:
NULL_SUBCARRIERS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 64, 65, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127]

In [3]:
connection = sqlite3.connect("../database/rti.db")

In [4]:
cursor = connection.cursor()

In [5]:
NULL_SUBCARRIERS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 64, 65, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127]

In [10]:
# Define a function to remove null subcarriers
def remove_null_subcarriers(csi_list):
    return [value for index, value in enumerate(csi_list) if index not in NULL_SUBCARRIERS]

# Path to the training-data directory
base_path = "../training-data"

# Iterate over each voxel folder in the training-data directory
for voxel_folder in os.listdir(base_path):
    voxel_path = os.path.join(base_path, voxel_folder)
    
    # Check if it's a directory
    if os.path.isdir(voxel_path):
        # Start a transaction
        cursor.execute("BEGIN TRANSACTION;")
        
        # Iterate over each activity file in the voxel folder
        for activity_file in glob.glob(os.path.join(voxel_path, "*.csv")):
            # Read the CSV file into a DataFrame
            data = pd.read_csv(activity_file)
            
            # Convert the 'csi' column to a list of lists
            data['csi'] = data['csi'].apply(eval).apply(remove_null_subcarriers)
            
            # Calculate amplitude and phase using vectorized operations
            csi_array = np.array(data['csi'].tolist())
            amplitudes = np.sqrt(csi_array[:, ::2]**2 + csi_array[:, 1::2]**2)
            phases = np.arctan2(csi_array[:, ::2], csi_array[:, 1::2])
            
            # Create amplitude and phase DataFrames
            for i in range(amplitudes.shape[1]):
                data[f'subcarrier_{i+1}_amplitude'] = amplitudes[:, i]
                data[f'subcarrier_{i+1}_phase'] = phases[:, i]
            
            # Calculate vectors
            vectors_list = []
            for subcarrier in range(1, amplitudes.shape[1] + 1):
                amplitude_vector = data[f'subcarrier_{subcarrier}_amplitude'].groupby(data.index // 67).mean().tolist()
                phase_vector = data[f'subcarrier_{subcarrier}_phase'].groupby(data.index // 67).mean().tolist()
                vectors_list.append((subcarrier, amplitude_vector, phase_vector))
            
            # Extract voxel_identifier and activity_name
            voxel_identifier = str(data.iloc[0]['voxel_no'])
            activity_name =  str(data.iloc[0]['activity']) if str(data.iloc[0]['activity']) != 'nan' else 'None'
            
            # Insert vectors into the database
            for subcarrier, amplitude_vector, phase_vector in vectors_list:
                cursor.execute("""
                    INSERT INTO distance_vector (voxel, subcarrier, activity, amplitude_vector, phase_vector)
                    VALUES (?, ?, ?, ?, ?)
                """, (voxel_identifier, subcarrier, activity_name, json.dumps(amplitude_vector), json.dumps(phase_vector)))
        
        # Commit the transaction for the voxel folderam
        connection.commit()
        print(f'Committed for {activity_file}')

/var/folders/r8/m3g8wfxx51q2vc_tfm526t500000gn/T/ipykernel_2869/378300428.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'subcarrier_{i+1}_phase'] = phases[:, i]
/var/folders/r8/m3g8wfxx51q2vc_tfm526t500000gn/T/ipykernel_2869/378300428.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'subcarrier_{i+1}_amplitude'] = amplitudes[:, i]
/var/folders/r8/m3g8wfxx51q2vc_tfm526t500000gn/T/ipykernel_2869/378300428.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame

Committed for ../training-data/Voxel-7/Standing-Movement-2024-04-11-14-08-25.csv
Committed for ../training-data/Voxel-0/None-2024-04-11-14-32-28.csv


/var/folders/r8/m3g8wfxx51q2vc_tfm526t500000gn/T/ipykernel_2869/378300428.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'subcarrier_{i+1}_phase'] = phases[:, i]
/var/folders/r8/m3g8wfxx51q2vc_tfm526t500000gn/T/ipykernel_2869/378300428.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'subcarrier_{i+1}_amplitude'] = amplitudes[:, i]
/var/folders/r8/m3g8wfxx51q2vc_tfm526t500000gn/T/ipykernel_2869/378300428.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame

Committed for ../training-data/Voxel-9/Standing-Stationary-2024-04-11-14-06-26.csv


/var/folders/r8/m3g8wfxx51q2vc_tfm526t500000gn/T/ipykernel_2869/378300428.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'subcarrier_{i+1}_phase'] = phases[:, i]
/var/folders/r8/m3g8wfxx51q2vc_tfm526t500000gn/T/ipykernel_2869/378300428.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'subcarrier_{i+1}_amplitude'] = amplitudes[:, i]
/var/folders/r8/m3g8wfxx51q2vc_tfm526t500000gn/T/ipykernel_2869/378300428.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame

...

/var/folders/r8/m3g8wfxx51q2vc_tfm526t500000gn/T/ipykernel_2869/378300428.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'subcarrier_{i+1}_phase'] = phases[:, i]
/var/folders/r8/m3g8wfxx51q2vc_tfm526t500000gn/T/ipykernel_2869/378300428.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'subcarrier_{i+1}_amplitude'] = amplitudes[:, i]
/var/folders/r8/m3g8wfxx51q2vc_tfm526t500000gn/T/ipykernel_2869/378300428.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame

Committed for ../training-data/Voxel-5/Standing-Stationary-2024-04-11-14-03-05.csv


/var/folders/r8/m3g8wfxx51q2vc_tfm526t500000gn/T/ipykernel_2869/378300428.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'subcarrier_{i+1}_phase'] = phases[:, i]
/var/folders/r8/m3g8wfxx51q2vc_tfm526t500000gn/T/ipykernel_2869/378300428.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'subcarrier_{i+1}_amplitude'] = amplitudes[:, i]
/var/folders/r8/m3g8wfxx51q2vc_tfm526t500000gn/T/ipykernel_2869/378300428.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame

Committed for ../training-data/Voxel-2/Standing-Movement-2024-04-11-13-54-48.csv


/var/folders/r8/m3g8wfxx51q2vc_tfm526t500000gn/T/ipykernel_2869/378300428.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'subcarrier_{i+1}_phase'] = phases[:, i]
/var/folders/r8/m3g8wfxx51q2vc_tfm526t500000gn/T/ipykernel_2869/378300428.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'subcarrier_{i+1}_amplitude'] = amplitudes[:, i]
/var/folders/r8/m3g8wfxx51q2vc_tfm526t500000gn/T/ipykernel_2869/378300428.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame